![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FFramework+Workflows%2FPyTorch%2Fserving&file=dataflow-setup.ipynb)
<!--- header table --->
<table align="left">
<tr>
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/Framework%20Workflows/PyTorch/serving/dataflow-setup.ipynb">
      <img width="32px" src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/Framework%20Workflows/PyTorch/serving/dataflow-setup.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FFramework%2520Workflows%2FPyTorch%2Fserving%2Fdataflow-setup.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/bigquery/import?url=https://github.com/statmike/vertex-ai-mlops/blob/main/Framework%20Workflows/PyTorch/serving/dataflow-setup.ipynb">
      <img width="32px" src="https://www.gstatic.com/images/branding/gcpiconscolors/bigquery/v1/32px.svg" alt="BigQuery logo">
      <br>Open in<br>BigQuery Studio
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/Framework%20Workflows/PyTorch/serving/dataflow-setup.ipynb">
      <img width="32px" src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</tr>
<tr>
  <td colspan="5" style="text-align: right">
    <b>Share This On: </b> 
    <a href="https://www.linkedin.com/sharing/share-offsite/?url=https://github.com/statmike/vertex-ai-mlops/blob/main/Framework%2520Workflows/PyTorch/serving/dataflow-setup.ipynb"><img src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="Linkedin Logo" width="20px"></a> 
    <a href="https://reddit.com/submit?url=https://github.com/statmike/vertex-ai-mlops/blob/main/Framework%2520Workflows/PyTorch/serving/dataflow-setup.ipynb"><img src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit Logo" width="20px"></a> 
    <a href="https://bsky.app/intent/compose?text=https://github.com/statmike/vertex-ai-mlops/blob/main/Framework%2520Workflows/PyTorch/serving/dataflow-setup.ipynb"><img src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="BlueSky Logo" width="20px"></a> 
    <a href="https://twitter.com/intent/tweet?url=https://github.com/statmike/vertex-ai-mlops/blob/main/Framework%2520Workflows/PyTorch/serving/dataflow-setup.ipynb"><img src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X (Twitter) Logo" width="20px"></a> 
  </td>
</tr>
<tr>
  <td colspan="5" style="text-align: right">
    <b>Connect With Author On: </b> 
    <a href="https://www.linkedin.com/in/statmike"><img src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="Linkedin Logo" width="20px"></a>
    <a href="https://www.github.com/statmike"><img src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub Logo" width="20px"></a> 
    <a href="https://www.youtube.com/@statmike-channel"><img src="https://upload.wikimedia.org/wikipedia/commons/f/fd/YouTube_full-color_icon_%282024%29.svg" alt="YouTube Logo" width="20px"></a>
    <a href="https://bsky.app/profile/statmike.bsky.social"><img src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="BlueSky Logo" width="20px"></a> 
    <a href="https://x.com/statmike"><img src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X (Twitter) Logo" width="20px"></a>
  </td>
</tr>
</table>

# Dataflow Infrastructure Setup

This notebook performs one-time setup for all Dataflow inference workflows.

## What You'll Learn

This workflow covers:

1. **Upload Model Archive**: Upload `.mar` file from training to GCS (if not already there)
2. **Extract .pt File**: Extract the traced `.pt` model from `.mar` for RunInference
3. **Create BigQuery Tables**: Set up tables for batch and streaming results
4. **Create Pub/Sub Topics**: Set up topics and subscriptions for streaming

## Prerequisites

- **Completed** `../pytorch-autoencoder.ipynb` (created the `.mar` file)
  - Local `.mar` file: `../files/pytorch-autoencoder/pytorch_autoencoder.mar`
- Google Cloud project with required APIs enabled

## File Organization

This notebook uses isolated workspace to avoid conflicts with other serving workflows:

**Local Workspace:**
```
./files/dataflow/
├── pytorch_autoencoder.mar  # Downloaded temporarily for extraction
└── final_model_traced.pt    # Extracted, then uploaded to GCS
```

**GCS Structure:**
```
gs://{PROJECT_ID}/frameworks/pytorch-autoencoder/dataflow/
├── pytorch_autoencoder.mar   # Model archive
└── final_model_traced.pt     # Traced model for RunInference
```

## Why This Setup?

**RunInference vs TorchServe:**
- **TorchServe** (Vertex Endpoints, Cloud Run, GCE, GKE) uses `.mar` files with custom handlers
- **RunInference** (Dataflow) loads PyTorch models directly from `.pt` files
- We extract the `.pt` from the `.mar` to use with RunInference

**One-Time Setup:**
- BigQuery tables persist across multiple pipeline runs
- Pub/Sub topics can be reused for streaming
- Model files stay in GCS for all Dataflow jobs

## Architecture

```
Local .mar file → Upload to GCS → Extract .pt → Upload .pt to GCS
                                                       ↓
                                      Used by all Dataflow pipelines
                                                       ↓
                                     ┌─────────────────┴──────────────────┐
                                     ↓                                    ↓
                             Batch RunInference              Streaming RunInference
                                     ↓                                    ↓
                               BigQuery Results                  Pub/Sub + BigQuery
```

---
## Environment Setup

This section will authenticate your session, enable required Google Cloud APIs, and install necessary Python packages.

**Package Installation Options (`REQ_TYPE`):**
- `PRIMARY`: Installs only the main packages
- `ALL` (Default): Installs exact versions of all packages and dependencies
- `COLAB`: Installs a Colab-optimized list

**Installation Tool Options (`INSTALL_TOOL`):**
- `pip` (Default): Standard Python package installer
- `uv`: Modern, fast Python package installer
- `poetry`: Dependency management tool

> **Note:** If running in Google Colab, the script will automatically set `REQ_TYPE = 'COLAB' to prevent package conflicts.

### Set Your Project ID

⚠️ **Action Required:** Replace the `PROJECT_ID` value below with your Google Cloud project ID before running this cell.

In [1]:
PROJECT_ID = 'statmike-mlops-349915' # replace with GCP project ID
REQ_TYPE = 'ALL' # Specify PRIMARY or ALL or COLAB
INSTALL_TOOL = 'poetry' # Specify pip, uv, or poetry

### Configuration

This cell defines the requirements files and Google Cloud APIs needed for this notebook.

In [2]:
REQUIREMENTS_URL = 'https://raw.githubusercontent.com/statmike/vertex-ai-mlops/refs/heads/main/Framework%20Workflows/PyTorch/requirements.txt'

REQUIRED_APIS = [
    "dataflow.googleapis.com",
    "bigquery.googleapis.com",
    "pubsub.googleapis.com",
    "storage.googleapis.com"
]

### Run Setup

This cell downloads the centralized setup code and configures your environment.

In [3]:
import os, urllib.request

# Download and import setup code
url = 'https://raw.githubusercontent.com/statmike/vertex-ai-mlops/refs/heads/main/core/notebook-template/python_setup.py'
urllib.request.urlretrieve(url, 'python_setup_local.py')
import python_setup_local as python_setup
os.remove('python_setup_local.py')

# Run setup
setup_info = python_setup.setup_environment(PROJECT_ID, REQ_TYPE, REQUIREMENTS_URL, REQUIRED_APIS, INSTALL_TOOL)


PYTHON GCP ENVIRONMENT SETUP

AUTHENTICATION
Checking for existing ADC...
✅ Existing ADC found.
✅ Project is correctly set to 'statmike-mlops-349915'.

API CHECK & ENABLE
✅ dataflow.googleapis.com is already enabled.
✅ bigquery.googleapis.com is already enabled.
✅ pubsub.googleapis.com is already enabled.
✅ storage.googleapis.com is already enabled.

PACKAGE MANAGEMENT
Installation Tool: poetry
✅ Found poetry at: /usr/local/google/home/statmike/.local/bin/poetry
✅ Running in poetry environment: /usr/local/google/home/statmike/.cache/pypoetry/virtualenvs/frameworks-pytorch-0KVJlKeQ-py3.13
ℹ️  Poetry mode: Installing from pyproject.toml (REQUIREMENTS_URL ignored)
✅ Found pyproject.toml at: /usr/local/google/home/statmike/Git/vertex-ai-mlops/Framework Workflows/PyTorch/pyproject.toml
   Changed working directory to: /usr/local/google/home/statmike/Git/vertex-ai-mlops/Framework Workflows/PyTorch
Running poetry install...
   Restored working directory to: /usr/local/google/home/statmike/Gi

---
## Python Setup

### Imports

In [4]:
import subprocess
import json
import os
import zipfile
from pathlib import Path
from google.cloud import bigquery
from google.cloud import storage
from google.cloud import pubsub_v1

### Variables - User Set

In [5]:
REGION = 'us-central1'
SERIES = 'frameworks'
EXPERIMENT = 'pytorch-autoencoder'

### Variables - Auto Set

In [6]:
PROJECT_ID = subprocess.run(['gcloud', 'config', 'get-value', 'project'], capture_output=True, text=True, check=True).stdout.strip()
PROJECT_NUMBER = subprocess.run(['gcloud', 'projects', 'describe', PROJECT_ID, '--format=value(projectNumber)'], capture_output=True, text=True, check=True).stdout.strip()

print(f"\n{'='*50}\nGoogle Cloud Project Information\n{'='*50}\nPROJECT_ID     = {PROJECT_ID}\nPROJECT_NUMBER = {PROJECT_NUMBER}\n{'='*50}\n")


Google Cloud Project Information
PROJECT_ID     = statmike-mlops-349915
PROJECT_NUMBER = 1026793852137



### Configurations

In [7]:
# GCS configuration
BUCKET_NAME = PROJECT_ID
BUCKET_URI = f"gs://{BUCKET_NAME}"

# Local paths
LOCAL_MAR_PATH = "../files/pytorch-autoencoder/pytorch_autoencoder.mar"
WORK_DIR = "./files/dataflow"

# GCS paths (isolated for dataflow)
GCS_DATAFLOW_DIR = f"{BUCKET_URI}/{SERIES}/{EXPERIMENT}/dataflow"
GCS_MAR_PATH = f"{GCS_DATAFLOW_DIR}/pytorch_autoencoder.mar"
GCS_PT_PATH = f"{GCS_DATAFLOW_DIR}/final_model_traced.pt"

# BigQuery configuration
BQ_DATASET = SERIES.replace('-', '_')
BQ_TABLE_BATCH = f"{EXPERIMENT.replace('-', '_')}_batch_results"
BQ_TABLE_STREAMING = f"{EXPERIMENT.replace('-', '_')}_streaming_results"

# Pub/Sub configuration
PUBSUB_TOPIC_INPUT = f"{EXPERIMENT}-input"
PUBSUB_TOPIC_OUTPUT = f"{EXPERIMENT}-output"
PUBSUB_SUB_INPUT = f"{EXPERIMENT}-input-sub"
PUBSUB_SUB_OUTPUT = f"{EXPERIMENT}-output-sub"

# Dataflow configuration
DATAFLOW_STAGING = f"{BUCKET_URI}/dataflow/staging"
DATAFLOW_TEMP = f"{BUCKET_URI}/dataflow/temp"

print(f"Bucket: {BUCKET_URI}")
print(f"\nLocal workspace:")
print(f"  Working directory: {WORK_DIR}")
print(f"  Source .mar file: {LOCAL_MAR_PATH}")
print(f"\nGCS dataflow directory: {GCS_DATAFLOW_DIR}")
print(f"  .mar file will be at: {GCS_MAR_PATH}")
print(f"  .pt file will be at: {GCS_PT_PATH}")
print(f"\nBigQuery dataset: {BQ_DATASET}")
print(f"  Batch results table: {BQ_TABLE_BATCH}")
print(f"  Streaming results table: {BQ_TABLE_STREAMING}")

Bucket: gs://statmike-mlops-349915

Local workspace:
  Working directory: ./files/dataflow
  Source .mar file: ../files/pytorch-autoencoder/pytorch_autoencoder.mar

GCS dataflow directory: gs://statmike-mlops-349915/frameworks/pytorch-autoencoder/dataflow
  .mar file will be at: gs://statmike-mlops-349915/frameworks/pytorch-autoencoder/dataflow/pytorch_autoencoder.mar
  .pt file will be at: gs://statmike-mlops-349915/frameworks/pytorch-autoencoder/dataflow/final_model_traced.pt

BigQuery dataset: frameworks
  Batch results table: pytorch_autoencoder_batch_results
  Streaming results table: pytorch_autoencoder_streaming_results


### Initialize Clients

In [8]:
# Create local working directory
os.makedirs(WORK_DIR, exist_ok=True)
print(f"✅ Local workspace: {WORK_DIR}")

# Initialize clients
storage_client = storage.Client(project=PROJECT_ID)
bq_client = bigquery.Client(project=PROJECT_ID)
pubsub_publisher = pubsub_v1.PublisherClient()
pubsub_subscriber = pubsub_v1.SubscriberClient()

# Get GCS bucket
bucket = storage_client.bucket(BUCKET_NAME)

print(f"✅ Clients initialized")
print(f"   Project: {PROJECT_ID}")
print(f"   Region: {REGION}")
print(f"   Bucket: {BUCKET_URI}")

✅ Local workspace: ./files/dataflow
✅ Clients initialized
   Project: statmike-mlops-349915
   Region: us-central1
   Bucket: gs://statmike-mlops-349915


---
## Upload and Prepare Model for RunInference

Upload the model archive to GCS and extract the TorchScript `.pt` file for use with Dataflow RunInference.

### Upload .mar File to GCS

Upload the model archive from the training notebook to GCS for Dataflow workflows.

In [9]:
# Check if local .mar file exists
from pathlib import Path

local_mar = Path(LOCAL_MAR_PATH)

if not local_mar.exists():
    raise FileNotFoundError(
        f"Model archive not found at {LOCAL_MAR_PATH}\n"
        f"Please run ../pytorch-autoencoder.ipynb first to create the model archive"
    )

print(f"✅ Found local .mar file: {LOCAL_MAR_PATH}")
print(f"   Size: {local_mar.stat().st_size:,} bytes ({local_mar.stat().st_size / 1024:.2f} KB)")

# Check if already uploaded to GCS
mar_blob = bucket.blob(f"{SERIES}/{EXPERIMENT}/dataflow/pytorch_autoencoder.mar")

if mar_blob.exists():
    print(f"\n✅ .mar file already exists in GCS: {GCS_MAR_PATH}")
    print(f"   Size: {mar_blob.size:,} bytes ({mar_blob.size / 1024:.2f} KB)")
    print(f"   Skipping upload...")
else:
    # Upload to GCS
    print(f"\nUploading to GCS: {GCS_MAR_PATH}")
    mar_blob.upload_from_filename(str(local_mar))
    print(f"✅ Upload complete!")
    print(f"   Size: {mar_blob.size:,} bytes ({mar_blob.size / 1024:.2f} KB)")

✅ Found local .mar file: ../files/pytorch-autoencoder/pytorch_autoencoder.mar
   Size: 29,851 bytes (29.15 KB)

Uploading to GCS: gs://statmike-mlops-349915/frameworks/pytorch-autoencoder/dataflow/pytorch_autoencoder.mar
✅ Upload complete!
   Size: 29,851 bytes (29.15 KB)


### Extract .pt File from .mar

In [11]:
# Download .mar file from GCS to working directory
local_mar_download = f"{WORK_DIR}/pytorch_autoencoder.mar"

print(f"Downloading {GCS_MAR_PATH}...")
mar_blob.download_to_filename(local_mar_download)
print(f"✅ Downloaded to {local_mar_download}")

# Extract the .pt file from the .mar archive
with zipfile.ZipFile(local_mar_download, 'r') as zip_ref:
    # List contents
    print("\nContents of .mar file:")
    for name in zip_ref.namelist():
        print(f"  - {name}")
    
    # Extract the traced model to working directory
    zip_ref.extract('final_model_traced.pt', WORK_DIR)

local_pt_path = f"{WORK_DIR}/final_model_traced.pt"
print(f"\n✅ Extracted final_model_traced.pt to {WORK_DIR}")

# Check extracted file size
pt_size = os.path.getsize(local_pt_path)
print(f"   Extracted model size: {pt_size:,} bytes ({pt_size / 1024:.2f} KB)")

✅ Downloaded to ./files/dataflow/pytorch_autoencoder.mar

Contents of .mar file:
  - handler.py
  - final_model_traced.pt
  - MAR-INF/MANIFEST.json

✅ Extracted final_model_traced.pt to ./files/dataflow
   Extracted model size: 47,408 bytes (46.30 KB)


### Upload .pt File to GCS

In [12]:
# Upload .pt file to GCS for RunInference
pt_blob = bucket.blob(f"{SERIES}/{EXPERIMENT}/dataflow/final_model_traced.pt")
pt_blob.upload_from_filename(local_pt_path)

print(f"✅ Uploaded .pt file to GCS: {GCS_PT_PATH}")
print(f"   Size: {pt_blob.size:,} bytes ({pt_blob.size / 1024:.2f} KB)")

# Clean up local files in working directory
os.remove(local_mar_download)
os.remove(local_pt_path)
print(f"\n✅ Cleaned up local files in {WORK_DIR}")

✅ Uploaded .pt file to GCS: gs://statmike-mlops-349915/frameworks/pytorch-autoencoder/dataflow/final_model_traced.pt
   Size: 47,408 bytes (46.30 KB)

✅ Cleaned up local files in ./files/dataflow


---
## Create BigQuery Result Tables

Create tables to store results from batch and streaming Dataflow jobs.

### Check for Existing Dataset

In [13]:
# Check if dataset exists
dataset_id = f"{PROJECT_ID}.{BQ_DATASET}"

try:
    dataset = bq_client.get_dataset(dataset_id)
    print(f"✅ Dataset already exists: {dataset_id}")
except:
    # Create dataset
    dataset = bigquery.Dataset(dataset_id)
    dataset.location = REGION
    dataset = bq_client.create_dataset(dataset)
    print(f"✅ Created dataset: {dataset_id}")

✅ Dataset already exists: statmike-mlops-349915.frameworks


### Create Batch Results Table

In [14]:
# Define schema for batch results
batch_schema = [
    bigquery.SchemaField("instance_id", "STRING"),
    bigquery.SchemaField("anomaly_score", "FLOAT64"),
    bigquery.SchemaField("encoded", "FLOAT64", mode="REPEATED"),
    bigquery.SchemaField("timestamp", "TIMESTAMP"),
]

# Create or get table
batch_table_id = f"{dataset_id}.{BQ_TABLE_BATCH}"

try:
    table = bq_client.get_table(batch_table_id)
    print(f"✅ Batch results table already exists: {batch_table_id}")
except:
    table = bigquery.Table(batch_table_id, schema=batch_schema)
    table = bq_client.create_table(table)
    print(f"✅ Created batch results table: {batch_table_id}")

✅ Created batch results table: statmike-mlops-349915.frameworks.pytorch_autoencoder_batch_results


### Create Streaming Results Table

In [15]:
# Define schema for streaming results (same as batch)
streaming_schema = [
    bigquery.SchemaField("instance_id", "STRING"),
    bigquery.SchemaField("anomaly_score", "FLOAT64"),
    bigquery.SchemaField("encoded", "FLOAT64", mode="REPEATED"),
    bigquery.SchemaField("timestamp", "TIMESTAMP"),
    bigquery.SchemaField("window_start", "TIMESTAMP"),
    bigquery.SchemaField("window_end", "TIMESTAMP"),
]

# Create or get table
streaming_table_id = f"{dataset_id}.{BQ_TABLE_STREAMING}"

try:
    table = bq_client.get_table(streaming_table_id)
    print(f"✅ Streaming results table already exists: {streaming_table_id}")
except:
    table = bigquery.Table(streaming_table_id, schema=streaming_schema)
    table = bq_client.create_table(table)
    print(f"✅ Created streaming results table: {streaming_table_id}")

✅ Created streaming results table: statmike-mlops-349915.frameworks.pytorch_autoencoder_streaming_results


---
## Create Pub/Sub Topics and Subscriptions

Create Pub/Sub infrastructure for streaming inference.

### Create Input Topic

In [16]:
# Create input topic
input_topic_path = pubsub_publisher.topic_path(PROJECT_ID, PUBSUB_TOPIC_INPUT)

try:
    topic = pubsub_publisher.get_topic(request={"topic": input_topic_path})
    print(f"✅ Input topic already exists: {input_topic_path}")
except:
    topic = pubsub_publisher.create_topic(request={"name": input_topic_path})
    print(f"✅ Created input topic: {input_topic_path}")

✅ Created input topic: projects/statmike-mlops-349915/topics/pytorch-autoencoder-input


### Create Input Subscription

In [17]:
# Create subscription for input topic
input_sub_path = pubsub_subscriber.subscription_path(PROJECT_ID, PUBSUB_SUB_INPUT)

try:
    subscription = pubsub_subscriber.get_subscription(request={"subscription": input_sub_path})
    print(f"✅ Input subscription already exists: {input_sub_path}")
except:
    subscription = pubsub_subscriber.create_subscription(
        request={
            "name": input_sub_path,
            "topic": input_topic_path,
            "ack_deadline_seconds": 60
        }
    )
    print(f"✅ Created input subscription: {input_sub_path}")

✅ Created input subscription: projects/statmike-mlops-349915/subscriptions/pytorch-autoencoder-input-sub


### Create Output Topic

In [18]:
# Create output topic
output_topic_path = pubsub_publisher.topic_path(PROJECT_ID, PUBSUB_TOPIC_OUTPUT)

try:
    topic = pubsub_publisher.get_topic(request={"topic": output_topic_path})
    print(f"✅ Output topic already exists: {output_topic_path}")
except:
    topic = pubsub_publisher.create_topic(request={"name": output_topic_path})
    print(f"✅ Created output topic: {output_topic_path}")

✅ Created output topic: projects/statmike-mlops-349915/topics/pytorch-autoencoder-output


### Create Output Subscription

In [19]:
# Create subscription for output topic
output_sub_path = pubsub_subscriber.subscription_path(PROJECT_ID, PUBSUB_SUB_OUTPUT)

try:
    subscription = pubsub_subscriber.get_subscription(request={"subscription": output_sub_path})
    print(f"✅ Output subscription already exists: {output_sub_path}")
except:
    subscription = pubsub_subscriber.create_subscription(
        request={
            "name": output_sub_path,
            "topic": output_topic_path,
            "ack_deadline_seconds": 60
        }
    )
    print(f"✅ Created output subscription: {output_sub_path}")

✅ Created output subscription: projects/statmike-mlops-349915/subscriptions/pytorch-autoencoder-output-sub


---
## Verify Setup

Check that all resources are ready for Dataflow jobs.

In [20]:
print("="*60)
print("DATAFLOW SETUP VERIFICATION")
print("="*60)

print("\n✅ Model Files:")
print(f"   .mar file: {GCS_MAR_PATH}")
print(f"   .pt file:  {GCS_PT_PATH}")

print("\n✅ BigQuery Tables:")
print(f"   Dataset: {dataset_id}")
print(f"   Batch results: {batch_table_id}")
print(f"   Streaming results: {streaming_table_id}")

print("\n✅ Pub/Sub Resources:")
print(f"   Input topic: {input_topic_path}")
print(f"   Input subscription: {input_sub_path}")
print(f"   Output topic: {output_topic_path}")
print(f"   Output subscription: {output_sub_path}")

print("\n✅ Dataflow Locations:")
print(f"   Staging: {DATAFLOW_STAGING}")
print(f"   Temp: {DATAFLOW_TEMP}")

print("\n" + "="*60)
print("READY FOR DATAFLOW JOBS!")
print("="*60)

DATAFLOW SETUP VERIFICATION

✅ Model Files:
   .mar file: gs://statmike-mlops-349915/frameworks/pytorch-autoencoder/dataflow/pytorch_autoencoder.mar
   .pt file:  gs://statmike-mlops-349915/frameworks/pytorch-autoencoder/dataflow/final_model_traced.pt

✅ BigQuery Tables:
   Dataset: statmike-mlops-349915.frameworks
   Batch results: statmike-mlops-349915.frameworks.pytorch_autoencoder_batch_results
   Streaming results: statmike-mlops-349915.frameworks.pytorch_autoencoder_streaming_results

✅ Pub/Sub Resources:
   Input topic: projects/statmike-mlops-349915/topics/pytorch-autoencoder-input
   Input subscription: projects/statmike-mlops-349915/subscriptions/pytorch-autoencoder-input-sub
   Output topic: projects/statmike-mlops-349915/topics/pytorch-autoencoder-output
   Output subscription: projects/statmike-mlops-349915/subscriptions/pytorch-autoencoder-output-sub

✅ Dataflow Locations:
   Staging: gs://statmike-mlops-349915/dataflow/staging
   Temp: gs://statmike-mlops-349915/dataflow

---
## Summary

In this notebook, you:

✅ Extracted `.pt` file from `.mar` for RunInference

✅ Created BigQuery tables for batch and streaming results

✅ Created Pub/Sub topics and subscriptions

✅ Verified all resources are ready

## Next Steps

You are now ready to run Dataflow inference jobs:

### 1. Batch Inference
Run [dataflow-batch-runinference.ipynb](./dataflow-batch-runinference.ipynb) to:
- Read transactions from BigQuery
- Apply RunInference with the PyTorch model
- Write anomaly scores back to BigQuery
- Analyze results

### 2. Streaming Inference
Run [dataflow-streaming-runinference.ipynb](./dataflow-streaming-runinference.ipynb) to:
- Read from Pub/Sub input topic
- Apply RunInference in real-time
- Write to Pub/Sub output topic and BigQuery
- Monitor streaming job

### 3. Vertex AI Endpoint Integration
Run [dataflow-vertex-endpoint.ipynb](./dataflow-vertex-endpoint.ipynb) to:
- Call Vertex AI Endpoint from Dataflow
- Compare performance vs RunInference
- Understand trade-offs

## Resources

- [Dataflow Documentation](https://cloud.google.com/dataflow/docs)
- [Apache Beam RunInference](https://beam.apache.org/documentation/ml/about-ml/)
- [PyTorch RunInference](https://beam.apache.org/documentation/ml/pytorch-inference/)
- [Pub/Sub Documentation](https://cloud.google.com/pubsub/docs)
- [BigQuery Documentation](https://cloud.google.com/bigquery/docs)